In [1]:
!pip install pyarrow==15.0.2
!pip install transformers
!pip install -q -U datasets einops wandb
!pip install -q accelerate peft bitsandbytes trl
!pip install -q huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.6 MB/s eta 0:00:00


In [2]:
# @title Importing relevant libraries, files, classes and functions
import os
import sys
from google.colab import drive, userdata
drive.mount('/content/drive')
base = '/content/drive/MyDrive/ABSA-LLMs-DistillingSbS'
sys.path.append(base)
import pandas as pd
import json
from huggingface_hub import login
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq
)
from train_utils import train_and_evaluate
from postprocessing_utils import inference_on_gpu
from data_utils import load_from_source, print_review_row_nicely

Mounted at /content/drive


SyntaxError: unterminated string literal (detected at line 56) (train_utils.py, line 56)

In [9]:
from data_utils import print_review_row_nicely

In [5]:
hf_token = userdata.get('HuggingFace_TOKEN')
login(token=hf_token, add_to_git_credential=True)
path_reviews = base + "/data/filtered_and_shuffled_reviews_35k.csv"
reviews = pd.read_csv(path_reviews)
dataset_train, dataset_valid, dataset_test = load_from_source()

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


The following cells finetune/distill a t5-large model to be good at outputting ABSA labels.

In [ ]:
base_model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(base_model_name, model_max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# @title tokenize_function_distilling_sbs_absa
def tokenize_function_distilling_sbs_absa(examples):
    def remove_braces(input_string):
        return input_string.replace("{", "").replace("}", "")

    model_inputs = tokenizer([f'Perform aspect-based sentiment analysis on the review of the app {app_name} and return the aspects and their corresponding sentiment polarities ("positive"/"negative"/"neutral"): "{review}"'  for  (app_name, review) in  zip(examples['appName'], examples['review'])], max_length=512, truncation=True, padding="max_length")
    expl_model_inputs = tokenizer([f'Perform aspect-based sentiment analysis on the review of the app {app_name} and return the aspects and explain what prompted each aspects inclusion: "{review}"' for  (app_name, review) in  zip(examples['appName'], examples['review'])], max_length=512, truncation=True, padding="max_length")
    model_inputs['expl_input_ids'] = expl_model_inputs['input_ids']
    model_inputs['expl_attention_mask'] = expl_model_inputs['attention_mask']

    with tokenizer.as_target_tokenizer():
        label_output_encodings = tokenizer([remove_braces(example) for example in examples['ABSALabels']], max_length=512, truncation=True, padding="max_length")
        rationale_output_encodings = tokenizer([remove_braces(example) for example in examples['ABSARationales']], max_length=512, truncation=True, padding="max_length")

    model_inputs['labels'] = label_output_encodings['input_ids']
    model_inputs['aux_labels'] = rationale_output_encodings['input_ids']

    return model_inputs

In [ ]:
# @title tokenize_function_standard_absa
def tokenize_function_standard_absa(examples):
    def remove_braces(input_string):
        return input_string.replace("{", "").replace("}", "")

    model_inputs = tokenizer([f'Perform aspect-based sentiment analysis on the review of the app {app_name} and return the aspects and their corresponding sentiment polarities ("positive"/"negative"/"neutral"): "{review}"'  for  (app_name, review) in  zip(examples['appName'], examples['review'])], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([remove_braces(example) for example in examples['ABSALabels']], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
training_method = 'distillingSbS' # change to 'standard' for standard ABSA
if training_method == 'distillingSbS':
    tokenize_function = tokenize_function_distilling_sbs_absa
elif training_method == 'standard':
    tokenize_function = tokenize_function_standard_absa

In [ ]:
tokenized_train = dataset_train.map(
    tokenize_function,
    batched=True,
)
tokenized_valid = dataset_valid.map(
    tokenize_function,
    batched=True,
)
tokenized_test = dataset_test.map(
    tokenize_function,
    batched=True,
)
tokenized_train = tokenized_train.remove_columns(['userReviewId', 'appName', 'review', 'ABSALabels', 'ABSARationales'])
tokenized_valid = tokenized_valid.remove_columns(['userReviewId', 'appName', 'review', 'ABSALabels', 'ABSARationales'])
tokenized_test = tokenized_test.remove_columns(['userReviewId', 'appName', 'review', 'ABSALabels', 'ABSARationales'])

Map:   0%|          | 0/24448 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5239 [00:00<?, ? examples/s]

Map:   0%|          | 0/5239 [00:00<?, ? examples/s]

In [ ]:
(history, trainer) = train_and_evaluate(0, tokenizer, tokenized_train, tokenized_valid, tokenized_test, training_method)

Evaluating the annotated labels for a small sample of reviews

In [10]:
path_reviews_sample = base + "/data/review_samples_annotation_eval.csv"
reviews_sample = pd.read_csv(path_reviews_sample) # 110 reviews with eval questions

In [7]:
# @title Eval questions in a python dict
eval_questions_absa ={

'Aspect Coverage' : {'Q' : 'Does the label correctly identify all relevant aspects mentioned in the review?', 'A' : None},

'Sentiment Accuracy': {'Q': 'Does the sentiment label (positive, negative, neutral) correctly reflect the sentiment expressed toward each aspect?', 'A': None},

'Aspect-Opinion Pairing': {'Q': 'Are the aspect and its associated opinion correctly paired (i.e., does the sentiment apply to the correct aspect)?', 'A': None},

'Label Granularity': {'Q': 'Is the level of detail in the aspect labels appropriate for the review (neither too broad nor too specific)?', 'A': None},

'Consistency with Similar Reviews': {'Q' : 'Is the labeling consistent with similar reviews (same aspects and sentiments labeled similarly across comparable reviews)?', 'A' : None},

'Clarity of Sentiment': {'Q' : 'Is the sentiment toward the aspect clear and unambiguous in the review text?', 'A' : None},

'Redundancy Check': {'Q': 'Are there any redundant or duplicate aspect-sentiment labels for this review?', 'A': None},

'False Positives/Negatives' : {'Q': 'Are there any aspects or sentiments labeled that are not actually present in the review (false positives) or any that are missing (false negatives)?' , 'A': None}

 }

In [8]:
eval_questions_per_id = {user_review_id : eval_questions_absa for user_review_id in reviews_sample['userReviewId'].tolist()}

In [ ]:
for user_review_id in eval_questions_per_id:
  print_review_row_nicely(reviews_sample.loc[reviews_sample['userReviewId']==user_review_id].squeeze())
  for q in eval_questions_absa:
    print(q)
    print(eval_questions_per_id[user_review_id][q]['Q'])
    eval_questions_per_id[user_review_id][q]['A'] = input()
  reviews_sample.loc[reviews_sample['userReviewId'] == user_review_id, 'eval_questions_absa'] = json.dumps(eval_questions_per_id[user_review_id])

In [ ]:
reviews_sample.to_csv(path_reviews_sample, index=False)